In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install category_encoders

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import seaborn as sns

SEED = 42

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [ ]:
cd gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.columns

Index(['ID', 'Price', 'Levy', 'Manufacturer', 'Model', 'Prod. year',
       'Category', 'Leather interior', 'Fuel type', 'Engine volume', 'Mileage',
       'Cylinders', 'Gear box type', 'Drive wheels', 'Doors', 'Wheel', 'Color',
       'Airbags'],
      dtype='object')

In [ ]:
len(train)

19237

In [ ]:
train.drop_duplicates(inplace = True)

In [ ]:
len(train)

18924

In [ ]:
train.drop(["ID", "Manufacturer"], axis = 1, inplace = True)

In [ ]:
train.head()

,Price,Levy,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,13328,1399,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,16621,1018,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,8467,-,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,3607,862,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,11726,446,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4


In [ ]:
train.dtypes

Price                 int64
Levy                 object
Model                object
Prod. year            int64
Category             object
Leather interior     object
Fuel type            object
Engine volume        object
Mileage              object
Cylinders           float64
Gear box type        object
Drive wheels         object
Doors                object
Wheel                object
Color                object
Airbags               int64
dtype: object

In [ ]:
train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')


test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

In [ ]:
lb = LabelEncoder()

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
train["Mileage"] = train["Mileage"].astype("str").astype("int") 
test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
test["Mileage"] = test["Mileage"].astype("str").astype("int")

In [ ]:
X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

In [ ]:
regressor = RandomForestRegressor(random_state = SEED)
regressor.fit(X_train, Y_train)

predicted = regressor.predict(X_test)
check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



The np.sqrt is : 1.0621995100249675
The r2 score is : -3.0239479017128623


In [ ]:
estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)

check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[16:56:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



[16:57:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.093467524897885
The r2 score is : -34.102103421507074


In [ ]:
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[16:57:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:57:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.2857713143423204
The r2 score is : -116.2448386971482


In [ ]:
train.drop(["Mileage"], axis = 1, inplace = True)

In [ ]:
X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[16:58:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.2424651076024316
The r2 score is : 0.5729589446857358


In [ ]:
train.drop(["Drive wheels"], axis = 1, inplace = True)

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[16:58:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:58:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.257905755990164
The r2 score is : 0.37276649277627183


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID"], axis = 1, inplace = True)
test.drop(["ID"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
train["Mileage"] = train["Mileage"].astype("str").astype("int") 
test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)

check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[16:59:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:59:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.2448956093533732
The r2 score is : -121.24987398042214


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[17:00:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.1852814243298577
The r2 score is : 0.3456353566543279


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID", "Mileage", "Manufacturer", "Cylinders", "Gear box type"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer", "Cylinders", "Gear box type"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

#train["Gear box type"] = lb.fit_transform(train["Gear box type"])
#test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

#train["Cylinders"] = train["Cylinders"].astype(int)
#test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[17:00:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:00:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.2605722558217434
The r2 score is : -397.4833811881592


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID", "Mileage", "Manufacturer", "Cylinders"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer", "Cylinders"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

#train["Cylinders"] = train["Cylinders"].astype(int)
#test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[17:02:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:02:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:02:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:02:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:02:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:02:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.1996722759567418
The r2 score is : -14.37403680416477


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[17:06:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:06:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:06:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:06:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:06:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:06:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.1852814243298577
The r2 score is : 0.3456353566543279


In [ ]:
#ID, Manufacturer, Mileage --> 0.57 deleted.


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

train["Model"] = lb.fit_transform(train["Model"])
test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[17:10:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:10:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.1852814243298577
The r2 score is : 0.3456353566543279


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
len(train)

19237

In [ ]:
train.drop_duplicates(inplace = True)
len(train)

18924

In [ ]:
train.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop_duplicates(inplace = True)

train.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()

encoder = ce.OneHotEncoder(cols=["Model"], handle_unknown='return_nan',return_df=True,use_cat_names=True)
train = encoder.fit_transform(train)
test = encoder.fit_transform(test)

#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

#train["Model"] = lb.fit_transform(train["Model"])
#test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"].astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor()),
            
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X_train, Y_train)
predicted = reg.predict(X_test)


check1 = np.sqrt(mean_squared_log_error(Y_test, predicted))
r2 = r2_score(Y_test, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning:

is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[18:07:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:17:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:18:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:18:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:19:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:19:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.2171310791822365
The r2 score is : 0.6152580685094706


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

train.drop_duplicates(inplace = True)

train.drop(["ID", "Mileage", "Manufacturer", "Model"], axis = 1, inplace = True)
test.drop(["ID", "Mileage", "Manufacturer", "Model"], axis = 1, inplace = True)

train['Engine volume'] = train['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
train['Engine volume'] = train['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

test['Engine volume'] = test['Engine volume'].replace(['2.0 Turbo', '2.2 Turbo','3.0 Turbo','1.4 Turbo','1.5 Turbo', '1.6 Turbo','2.3 Turbo','2.8 Turbo','1.8 Turbo', '2.4 Turbo', '3.5 Turbo', '3.2 Turbo','1.3 Turbo','2.5 Turbo','1.9 Turbo', '4.4 Turbo', '4.7 Turbo', '0.2 Turbo','4.8 Turbo', '4.6 Turbo','1.2 Turbo','1.7 Turbo', '6.3 Turbo', '2.7 Turbo','2.9 Turbo', '4.0 Turbo','3.6 Turbo','3.7 Turbo','5.5 Turbo','2.1 Turbo','0.7 Turbo', '0.6 Turbo','1.0 Turbo', '4.5 Turbo', '0.8 Turbo', '4.2 Turbo', '3.1', '5.0 Turbo','5.7 Turbo','0.4 Turbo', '5.4 Turbo', '0.3 Turbo','1.1 Turbo', '6.0 Turbo', '2.6 Turbo'],'Turbo')
test['Engine volume'] = test['Engine volume'].replace(['3.5','3','1.3','2.5','2','1.8','2.4','4','1.6','3.3','4.7','1.5','4.4','3.6','2.3','2.2','1.4','5.5','3.2','3.8','4.6','1.2','5', '1.7', '2.9', '0.5','1.9','2.7','4.8','5.3','0.4','2.8','1.1','2.1','0.7','5.4','3.7','1','2.6','5.7','3.4','4.3','4.2','5.9','6.8','4.5','0.6','7.3','0.1','6.3','6.4','5.2','5.8','0.8', '6.7', '6.2', '0', '20', '0.3', '0.2','5.6', '6', '3.9', '0.9', '6.6', '6.1', '10.8'],'Non-Turbo')

lb = LabelEncoder()


#train["Manufacturer"] = lb.fit_transform(train["Manufacturer"])
#test["Manufacturer"] = lb.fit_transform(test["Manufacturer"])

train["Leather interior"] = lb.fit_transform(train["Leather interior"])
test["Leather interior"] = lb.fit_transform(test["Leather interior"])

train["Drive wheels"] = lb.fit_transform(train["Drive wheels"])
test["Drive wheels"] = lb.fit_transform(test["Drive wheels"])

train["Doors"] = lb.fit_transform(train["Doors"])
test["Doors"] = lb.fit_transform(test["Doors"])

train["Wheel"] = lb.fit_transform(train["Wheel"])
test["Wheel"] = lb.fit_transform(test["Wheel"])

train["Color"] = lb.fit_transform(train["Color"])
test["Color"] = lb.fit_transform(test["Color"])
 
train["Engine volume"] = lb.fit_transform(train["Engine volume"])
test["Engine volume"] = lb.fit_transform(test["Engine volume"])

train["Gear box type"] = lb.fit_transform(train["Gear box type"])
test["Gear box type"] = lb.fit_transform(test["Gear box type"])

train["Fuel type"] = lb.fit_transform(train["Fuel type"])
test["Fuel type"] = lb.fit_transform(test["Fuel type"])

train["Category"] = lb.fit_transform(train["Category"])
test["Category"] = lb.fit_transform(test["Category"])

#train["Model"] = lb.fit_transform(train["Model"])
#test["Model"] = lb.fit_transform(test["Model"])

train['Levy'] = train['Levy'].replace("-", "0")
test['Levy'] = test['Levy'].replace("-", "0")
train["Levy"] = train["Levy"].astype("str").astype("int")
test["Levy"] = test["Levy"].astype("str").astype("int")

train["Cylinders"] = train["Cylinders"].astype(int)
test["Cylinders"] = test["Cylinders"].astype(int)

#train[['Mileage']] = train[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#train["Mileage"] = train["Mileage"].astype("str").astype("int") 
#test[['Mileage']] = test[['Mileage']].replace(to_replace=r'([a-z/]+|[A-Z/]+)', value='', regex=True)
#test["Mileage"] = test["Mileage"]  .astype("str").astype("int")

X = train.iloc[ : ,  1: ]
Y = train.iloc[ : , : 1]

#X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = SEED, shuffle = True)

sc = StandardScaler()
X = sc.fit_transform(X)

#X_train = sc.fit_transform(X_train)
#X_test = sc.fit_transform(X_test)


estimators = [
              ("xgb", RandomForestRegressor(random_state = SEED)),
              ("svr", LinearSVR(random_state = SEED)),
              ("rf", xgb.XGBRegressor())           
]

reg = StackingRegressor(
    estimators = estimators, 
    final_estimator = RandomForestRegressor(random_state = SEED)
)

reg.fit(X, Y)

predicted = reg.predict(X)

check1 = np.sqrt(mean_squared_log_error(Y, predicted))
r2 = r2_score(Y, predicted)

print(f"The np.sqrt is : {check1}")
print(f"The r2 score is : {r2}")

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_stacking.py:642: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



[19:23:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:23:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:23:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:24:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
The np.sqrt is : 1.0376915991989972
The r2 score is : -0.11667664630419572


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
len(train) + len(test)

27482

In [ ]:
len(train)

19237

In [ ]:
len(test)

8245

In [ ]:
dataset = [train, test]

In [ ]:
result = pd.concat(dataset)

In [ ]:
result

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328.0,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621.0,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467.0,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607.0,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726.0,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8240,44824410,NaN,1357,FIAT,Stilo,2004,Universal,No,Petrol,1.6,164000 km,4.0,Manual,Front,04-May,Left wheel,Blue,6
8241,45682346,NaN,-,BMW,528 F10,2012,Sedan,Yes,Petrol,2.8 Turbo,156000 km,4.0,Tiptronic,Front,04-May,Left wheel,Black,6
8242,45649860,NaN,1646,LEXUS,GX 460,2015,Jeep,Yes,Petrol,4.6,102907 km,8.0,Automatic,4x4,04-May,Left wheel,Black,0
8243,45809706,NaN,784,TOYOTA,Aqua,2017,Hatchback,Yes,Petrol,1.5,3000 km,4.0,Automatic,Front,04-May,Left wheel,Green,4


In [ ]:
train

,ID,Price,Levy,Manufacturer,Model,Prod. year,Category,Leather interior,Fuel type,Engine volume,Mileage,Cylinders,Gear box type,Drive wheels,Doors,Wheel,Color,Airbags
0,45654403,13328,1399,LEXUS,RX 450,2010,Jeep,Yes,Hybrid,3.5,186005 km,6.0,Automatic,4x4,04-May,Left wheel,Silver,12
1,44731507,16621,1018,CHEVROLET,Equinox,2011,Jeep,No,Petrol,3,192000 km,6.0,Tiptronic,4x4,04-May,Left wheel,Black,8
2,45774419,8467,-,HONDA,FIT,2006,Hatchback,No,Petrol,1.3,200000 km,4.0,Variator,Front,04-May,Right-hand drive,Black,2
3,45769185,3607,862,FORD,Escape,2011,Jeep,Yes,Hybrid,2.5,168966 km,4.0,Automatic,4x4,04-May,Left wheel,White,0
4,45809263,11726,446,HONDA,FIT,2014,Hatchback,Yes,Petrol,1.3,91901 km,4.0,Automatic,Front,04-May,Left wheel,Silver,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19232,45798355,8467,-,MERCEDES-BENZ,CLK 200,1999,Coupe,Yes,CNG,2.0 Turbo,300000 km,4.0,Manual,Rear,02-Mar,Left wheel,Silver,5
19233,45778856,15681,831,HYUNDAI,Sonata,2011,Sedan,Yes,Petrol,2.4,161600 km,4.0,Tiptronic,Front,04-May,Left wheel,Red,8
19234,45804997,26108,836,HYUNDAI,Tucson,2010,Jeep,Yes,Diesel,2,116365 km,4.0,Automatic,Front,04-May,Left wheel,Grey,4
19235,45793526,5331,1288,CHEVROLET,Captiva,2007,Jeep,Yes,Diesel,2,51258 km,4.0,Automatic,Front,04-May,Left wheel,Black,4
